In [3]:
# Import packages.
import cvxpy as cp
import numpy as np
import scipy.stats as stats

In [ ]:
import pickle
import os


print(os.path.dirname(os.getcwd())+'/Prepare_Datasets/indicators.pkl')
# stocks = pickle.load(open(, 'rb'))
samples = list()
for i in stocks:
    samples.append(stocks[i])
samples = np.asarray(samples, dtype=np.float64)
print(samples)

/home/eeuser/Desktop/ELEC5470_Assignments/Final_ProjectPrepare_Datasets/indicators.pkl


NameError: name 'stocks' is not defined

In [4]:
# N assets
N = 10
# S sample
S = 100
# Sigma
Sig = np.eye(N)
# Sigma^{1/2}
Sig2 = np.linalg.cholesky(Sig)
print(np.allclose(Sig, Sig2 @ Sig2.T)) 
# muhat
mu = np.zeros((N,1))
#confidence level alpha
alpha = 0.05
# standard norm quantile
z_95 = stats.norm.ppf(1-alpha)
z_975 = stats.norm.ppf(1-alpha/2)
# delta^2 = chi^2 quantile
delta2 = stats.chi2.ppf(1-alpha, df=N)
# lambda0
lambda0 = 0.5
# beta
beta = 0.5


True


In [5]:
# Box uncertainty set
A = np.hstack((Sig2, np.zeros((N,1))))
c = mu + (1-2*beta)*(z_975/np.sqrt(S))*Sig2@np.ones((N,1))
c = np.append(c, 1)
c = c/((1-lambda0)*z_95)
e = np.ones(N+1)
e[-1] = 0

In [20]:
# set random seed
np.random.seed(0)
# objective function coeficient
f = np.zeros(N + 1)  
f[-1] = 1      

# define variables x=[w,t]^T
x = cp.Variable(N+1)
# SOC constraints
soc_constraints = [cp.SOC(c.T @ x , A @ x )]
# Define and solve the CVXPY problem.
prob = cp.Problem(cp.Minimize(f.T@x),
                  soc_constraints + [e.T @ x == 1])

prob.solve()
# Print result.
print("The optimal value:", prob.value)
print("Optimal solution x:", x.value)

The optimal value: 0.2600741922463065
Optimal solution x: [0.1        0.1        0.1        0.1        0.1        0.1
 0.1        0.1        0.1        0.1        0.26007419]


In [26]:
A1 = np.hstack((Sig2, np.zeros((N,1))))
c1 = np.append(mu, 1)
a = (1-lambda0)*z_95 + (1-2*beta)*np.sqrt(delta2/S)
c1 = c1/a
e1 = np.ones(N+1)
e1[-1] = 0
print(e1)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]


In [28]:
# set random seed
np.random.seed(0)
# objective function coeficient
f1 = np.zeros(N + 1) 
f1[-1] = 1      
print(f1.shape)

# define variables x=[w,t]^T
x1 = cp.Variable(N+1)
# SOC constraints
soc_constraints1 = [cp.SOC(c1.T @ x1 , A1 @ x1 )]
# Define and solve the CVXPY problem.
prob1 = cp.Problem(cp.Minimize(f1.T@x1),
                  soc_constraints1 + [e1.T @ x1 == 1])

prob1.solve()
# Print result.
print("The optimal value:", prob1.value)
print("Optimal solution x:", x1.value)

(11,)
The optimal value: 0.2600741922463065
Optimal solution x: [0.1        0.1        0.1        0.1        0.1        0.1
 0.1        0.1        0.1        0.1        0.26007419]
